# YoloV4 Polyp Detection

Team 13: James Medel, Jay Bharadva, Sparsha Ramakrishna, Shreya Hunur, Priyank Jagad

May 12, 2022

We'll be training **Darknet YOLOv4** on our custom **Polyp dataset** (of polyp images and bounding box/classification labels) to perform **Polyp detection and classfication**.

## Train Polyp Detector using YOLOv4 in 12 Steps

If you get disconnected or lose your session, run steps 2, 5 and 6 again to edit makefile and build darknet every single time, otherwise darknet wont work.

## Step 1: Create `yolov4` and `training` folders in your project

Create ***yolov4*** folder in your project.

Inside ***yolov4*** folder, create ***training*** folder. We will save our trained weights.

## Step 2: Navigate to `yolov4` folder

In [4]:
# yolov4 folder will be in current directory
yolov4_basedir = "yolov4/"

## Step 3: Clone `darknet` git repo

`pip install gitpython`

Note: make sure **git** is installed.

In [1]:
from git import Repo

In [5]:
darknet_repo_dir = yolov4_basedir + "darknet"
Repo.clone_from("https://github.com/AlexeyAB/darknet", darknet_repo_dir)

<git.repo.base.Repo 'C:\\Users\\james\\Documents\\GitHub\\Lesion-Detection\\Notebooks\\yolov4\\darknet\\.git'>

## Step 4: Create Following Files Needed for Training a Polyp Detector

We'll walk through each of the following steps using Python:

- a). Convert PascalVOC to YOLO Labels for Polyp Dataset
- b). Customizing YOLOv4 cfg file (hyperparameters)
- c). Create `obj.data` and `obj.names` files
- d). Create `train.txt` and `valid.txt` Files for Training

### Step 4A: Convert PascalVOC to YOLO Labels for Polyp DataSet

We will later use this **Polyp training data** to train **Darknet YOLOv4**.

In [1]:
import sys
sys.path.append("../lib/prep/yolov4")

In [2]:
from convert_voc_to_yolo import ConvertPascalVocToYolo

In [1]:
polypset_base = "PolypsSet/"

In [3]:
polypset_dirs = [polypset_base + "train2019"]
polypset_classes = ["adenomatous", "hyperplastic"]
ConvertPascalVocToYolo(polypset_dirs, polypset_classes)

100%|███████████████████████████████████████████████████████████████████████████| 28773/28773 [02:30<00:00, 191.42it/s]


Finished processing: PolypsSet/train2019


***NOTE***: 

The Python Code **convert_voc_to_yolo** is based on this gist:

- https://gist.github.com/Amir22010/a99f18ca19112bc7db0872a36a03a1ec

That Python Code from that gist is originally based on Joseph Redmon's Darknet: voc_label.py:

- https://github.com/pjreddie/darknet/blob/master/scripts/voc_label.py

That Python Code has been modified for our **PolypsSet** folder structure and I also incorporated **ConvertPascalVocToYolo** class, so its easier to use in a Jupyter Notebook.

### Step 4B: Update YOLOv4 `config` file's Hyperparameters

We'll copy **yolov4-custom.cfg** from ***darknet/cfg*** directory, make changes to it and move it to ***yolov4*** folder in our project.

With Python, we'll make the following updates using **[pysed](https://pypi.org/project/pysed/)**:

`pip install pysed`

- change line batch to `batch=64`
- change line subdivision to `subdivisions=16`
- change line max_batches to (classes*2000 but not less than number of training images and not less than 6000), f.e. `max_batches=6000` if you tain for 3 classes. So since we have 2 Polyp classes, we'll set it to 6000.
- change line steps to **80%** and **90%** of **max_batches**, f.e. `steps=4800,5400`
- set network size `width=416` and `height=416` or any value **multiple of 32**
- change line **classes=80** to your `number of classes` in each of 3 **[yolo]-layers**. In our case `classes=2` in these 3 [yolo] layers.
- change **[filters=255]** to `filters=(classes + 5)x3` in the **3 [ convolutional] before each [yolo] layer**, keep inn minnd that it only has to be the **last [convolutional] before each of the [yolo] layers**. So if classes=1, then it should be filters=18. If **classes=2**, then **filters=21**.

You should only have to run the Python code in this step 4B one time since the **yolov4-custom.cfg** will be updated as needed.

In [5]:
import shutil, os

In [31]:
# set input and output files for updating YOLOv4 config file
yolo_dir = "yolov4/"
darknet_path = yolo_dir + "darknet/"

yolo_cfg_file = "yolov4-custom.cfg"
darknet_yolo_cfg_f = darknet_path + "cfg/" + yolo_cfg_file
out_yolo_cfg_f = yolo_dir + yolo_cfg_file

In [5]:
# Copy yolov4-custom.cfg to output dir
in_files = [darknet_yolo_cfg_f]
for f in in_files:
    shutil.copy(f, out_dir)

In [19]:
import shlex
from pysed import main as pysedmain

In [24]:
# Create sed replace text function
def sed_rep_txt(pattern, replacement, filename):
    cmd_line_args = '-r "{pattern}" "{replacement}" {filename}'.format(
        pattern=pattern, replacement=replacement, filename=filename)
    args = shlex.split(cmd_line_args)
    isWrite = True
    
    with open(filename, 'rU') as f:
        data = f.read()
        pysedmain.executeArguments(args, data, filename, isWrite)

In [25]:
# update batch to 64 in case its something different
sed_rep_txt("batch=64", "batch=64", out_yolo_cfg_f)

C:\Users\james\miniconda3\envs\yolov4-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys


In [27]:
# update subdivisions to 16 in case its something different
sed_rep_txt("subdivisions=16", "subdivisions=16", out_yolo_cfg_f)

C:\Users\james\miniconda3\envs\yolov4-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys


In [32]:
# update max_batches to 6000 since our num classes*2000 less than 6000
classes = len(polypset_classes)
if (classes*2000) < 6000:
    max_batches=6000
else:
    max_batches = classes*2000
    
new_max_batches = "max_batches={}".format(max_batches)
sed_rep_txt("max_batches = 500500", new_max_batches, out_yolo_cfg_f)

C:\Users\james\miniconda3\envs\yolov4-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys


In [34]:
# update steps to be 80% and 90% of max batches
pct_max_batches = [int(max_batches*0.8), int(max_batches*0.9)]
new_steps = "steps={},{}".format(pct_max_batches[0], pct_max_batches[1])
sed_rep_txt("steps=400000,450000", new_steps, out_yolo_cfg_f)

C:\Users\james\miniconda3\envs\yolov4-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys


In [36]:
# set network width and height to 416 or any multiple of 32
width=32*13 # 416
height=32*13

new_width = "width={}".format(width)
new_height = "height={}".format(height)

sed_rep_txt("width=608", new_width, out_yolo_cfg_f)
sed_rep_txt("height=608", new_height, out_yolo_cfg_f)

C:\Users\james\miniconda3\envs\yolov4-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys


In [37]:
# change line classes=80 to your `number of classes` in 
# each of 3 [yolo]-layers

num_polyp_classes = "classes={}".format(classes)
sed_rep_txt("classes=80", num_polyp_classes, out_yolo_cfg_f)

C:\Users\james\miniconda3\envs\yolov4-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys


In [39]:
# change filters=255 to `filters=(classes + 5)x3` in the
# 3 [ convolutional] layers before each [yolo] layer

num_filters = (classes + 5)*3
new_filters = "filters={}".format(num_filters)

sed_rep_txt("filters=255", new_filters, out_yolo_cfg_f)

C:\Users\james\miniconda3\envs\yolov4-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys


### Step 4C: Create `obj.data` and `obj.names` files

We'll create **obj.data** and **obj.names** files and put them into our ***yolov4*** folder.

In [43]:
obj_names_f = yolo_dir + "obj.names"

# TODO: Check if file exists, then delete it and replace it
with open(obj_names_f, 'a') as f:
    for polyp_class in polypset_classes:
        f.write("{}\n".format(polyp_class))

# check file after appending
with open(obj_names_f, "r") as f:
    print(f.read())

adenomatous
hyperplastic



In [45]:
# Create obj.data file in yolov4 folder
obj_data_f = yolo_dir + "obj.data"

# polypset_base has a "/" at end
obj_data_lines = ["classes={}".format(classes),
                  "train = {}train.txt".format(polypset_base),
                  "valid = {}valid.txt".format(polypset_base),
                  "names = {}{}".format(polypset_base, obj_names_f),
                  "backup = {}trained_weights".format(yolo_dir)]

# TODO: Check if file exists, then delete it and replace it
with open(obj_data_f, 'a') as f:
    for obj_data_line in obj_data_lines:
        f.write("{}\n".format(obj_data_line))

# check file after appending
with open(obj_data_f, "r") as f:
    print(f.read())

classes=2
train = PolypsSet/train.txt
valid = PolypsSet/valid.txt
names = PolypsSet/yolov4/obj.names
backup = yolov4/trained_weights



### Step 4D: Create `train.txt` and `valid.txt` Files for Training

***train.txt*** file has paths to about 85% of the Polyp images and
***test.txt*** file has paths t about 10% of the Polyp images.

In [2]:
def get_filepaths(basepath, remove_ext=False):
    files = []
    filenames = []
    for filename in os.listdir(basepath):
#         print("filename =", filename)
        if remove_ext is True:
            file_name, file_ext = filename.split(".")
#             print("file_name =", file_name)
#             print("file_ext =", file_ext)
            filepath = basepath + "/" + file_name
            files.append(filepath)
            filenames.append(file_name)
        else:
            filepath = basepath + "/" + filename
            files.append(filepath)
            filenames.append(filename)
    return files, filenames

In [51]:
train_X_basepath = "PolypsSet/train2019/Image"
file_type = ".jpg"
train_X_filepaths, train_X_filenames = get_filepaths(train_X_basepath, file_type)

In [52]:
train_X_filepaths[:5]

['PolypsSet/train2019/Image/1.jpg',
 'PolypsSet/train2019/Image/10.jpg',
 'PolypsSet/train2019/Image/100.jpg',
 'PolypsSet/train2019/Image/1000.jpg',
 'PolypsSet/train2019/Image/10000.jpg']

In [53]:
# Create train.txt file holding paths to all training images for YOLOv4
train_txt_f = "{}train.txt".format(polypset_base)

# TODO: Check if file exists, then delete it and replace it
with open(train_txt_f, 'a') as f:
    for train_img_filepath in train_X_filepaths:
        f.write("{}\n".format(train_img_filepath))

In [3]:
def get_filepaths_videonum_dirs_xy(X_basepath, y_basepath):
    videonum_X_filepaths = []
    videonum_X_filenames = []
    videonum_y_filepaths = []
    videonum_y_filenames = []
    for videonum_X_dir, videonum_y_dir in zip(os.listdir(X_basepath), os.listdir(y_basepath)):
        videonum_X_dirpath = X_basepath + "/" + videonum_X_dir
        videonum_y_dirpath = y_basepath + "/" + videonum_y_dir
        videonum_y_dirfilepaths_tmp = []
        videonum_y_dirfilenames_tmp = []
        videonum_X_dirfilepaths, videonum_X_dirfilenames = get_filepaths(videonum_X_dirpath, remove_ext=True)
        videonum_y_dirfilepaths, videonum_y_dirfilenames = get_filepaths(videonum_y_dirpath, remove_ext=True)
        if len(videonum_X_dirfilenames) != len(videonum_y_dirfilenames):
            for y_i in range(len(videonum_y_dirfilenames)):
                if videonum_y_dirfilenames[y_i] in videonum_X_dirfilenames:
                    videonum_y_dirfilenames_tmp.append(videonum_y_dirfilenames[y_i] + ".xml")
                    videonum_y_dirfilepaths_tmp.append(videonum_y_dirfilepaths[y_i] + ".xml")
            videonum_y_filepaths.extend(videonum_y_dirfilepaths_tmp)
            videonum_y_filenames.extend(videonum_y_dirfilenames_tmp)
        else:
            videonum_y_dirfilepaths = [filepath + ".xml" for filepath in videonum_y_dirfilepaths]
            videonum_y_dirfilenames = [filename + ".xml" for filename in videonum_y_dirfilenames]
            videonum_y_filepaths.extend(videonum_y_dirfilepaths)
            videonum_y_filenames.extend(videonum_y_dirfilenames)
        videonum_X_dirfilepaths = [filepath + ".jpg" for filepath in videonum_X_dirfilepaths]
        videonum_X_dirfilenames = [filename + ".jpg" for filename in videonum_X_dirfilenames]
        videonum_X_filepaths.extend(videonum_X_dirfilepaths)
        videonum_X_filenames.extend(videonum_X_dirfilenames)
    return videonum_X_filepaths, videonum_X_filenames, videonum_y_filepaths, videonum_y_filenames

In [6]:
valid_X_basepath = "PolypsSet/val2019/Image"
valid_y_basepath = "PolypsSet/val2019/Annotation"

# Problem is that the valid set has more labels than there are images, so need to make them equal
valid_X_filepaths, valid_X_filenames, valid_y_filepaths, valid_y_filenames= get_filepaths_videonum_dirs_xy(valid_X_basepath, valid_y_basepath)

In [7]:
valid_X_filepaths[:2]

['PolypsSet/val2019/Image/1/1.jpg', 'PolypsSet/val2019/Image/1/10.jpg']

In [8]:
valid_X_filepaths[1000:1003]

['PolypsSet/val2019/Image/13/103.jpg',
 'PolypsSet/val2019/Image/13/104.jpg',
 'PolypsSet/val2019/Image/13/105.jpg']

In [9]:
# Create valid.txt file holding paths to all validation images for YOLOv4
valid_txt_f = "{}valid.txt".format(polypset_base)

# TODO: Check if file exists, then delete it and replace it
with open(valid_txt_f, 'a') as f:
    for valid_img_filepath in valid_X_filepaths:
        f.write("{}\n".format(valid_img_filepath))